In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import timeit
import datetime
import xgboost as xgb
import math
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import plotly.graph_objs as go
import folium
import json
import utm
import scipy
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import joblib
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
import folium
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
import datetime
import scipy.stats as stats

# libraries for import all file names
from prettytable import PrettyTable
from os import listdir
from os.path import isfile, join
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import accuracy_score , mean_squared_error
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from yellowbrick.model_selection import FeatureImportances
from sklearn.linear_model import LinearRegression



# Funcions auxiliars

In [7]:
#Funció per poder mostrar i ocultar cel.les (per fer el notebook més llegible)
from IPython.display import HTML
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

# Visualització Geografica

In [26]:
#Funció visualització estacions SMC i AMC

def prepareMap():  
    m = folium.Map(location=[41.96867, 2.41404], zoom_start=10)

    tooltip = ''
    #Dades SMC
    folium.Marker(
        [42.07398, 2.20862], popup="<i>CC (Oris)</i>", tooltip=tooltip, icon=folium.Icon(icon="umbrella", prefix='fa')
    ).add_to(m)
    folium.Marker(
        [42.37717, 2.41456], popup="<i>CG (Molló - Fabert)</i>", tooltip=tooltip, icon=folium.Icon(icon="umbrella", prefix='fa')
    ).add_to(m)
    folium.Marker(
        [42.25839, 2.36429 ], popup="<i>CI (Sant Pau de Següries)</i>", tooltip=tooltip, icon=folium.Icon(icon="umbrella", prefix='fa')
    ).add_to(m)
    folium.Marker(
        [41.87813, 2.17873 ], popup="<i>CY (Muntanyola)</i>", tooltip=tooltip, icon=folium.Icon(icon="umbrella", prefix='fa')
    ).add_to(m)
    folium.Marker(
        [42.39848, 2.15517 ], popup="<i>DG (Nuria)</i>", tooltip=tooltip, icon=folium.Icon(icon="umbrella", prefix='fa')
    ).add_to(m)
    folium.Marker(
        [42.11653, 2.78969 ], popup="<i>DJ (Banyoles)</i>", tooltip=tooltip, icon=folium.Icon(icon="umbrella", prefix='fa')
    ).add_to(m)
    folium.Marker(
        [41.96095, 2.63108 ], popup="<i>DN (Anglès)</i>", tooltip=tooltip, icon=folium.Icon(icon="umbrella", prefix='fa')
    ).add_to(m)
    folium.Marker(
        [41.96867, 2.41404 ], popup="<i>KE (Pantà de Sau)</i>", tooltip=tooltip, icon=folium.Icon(icon="umbrella", prefix='fa')
    ).add_to(m)
    folium.Marker(
        [42.22189, 2.2427 ], popup="<i>M6 (Sant Joan de les Abadesses)</i>", tooltip=tooltip, icon=folium.Icon(icon="umbrella", prefix='fa')
    ).add_to(m)
    folium.Marker(
        [42.05398, 3.06195 ], popup="<i>UB ('La Tallada d'Empordà)</i>", tooltip=tooltip, icon=folium.Icon(icon="umbrella", prefix='fa')
    ).add_to(m)
    folium.Marker(
        [41.87449, 2.92694 ], popup="<i>UN (Cassà de la Selva)</i>", tooltip=tooltip, icon=folium.Icon(icon="umbrella", prefix='fa')
    ).add_to(m)
    folium.Marker(
        [41.91461, 2.82069 ], popup="<i>UO (Fornells de la Selva)</i>", tooltip=tooltip, icon=folium.Icon(icon="umbrella", prefix='fa')
    ).add_to(m)
    folium.Marker(
        [41.95224, 2.23271], popup="<i>V3 (Gurb)</i>", tooltip=tooltip, icon=folium.Icon(icon="umbrella", prefix='fa')
    ).add_to(m)
    folium.Marker(
        [42.11477, 2.21483 ], popup="<i>V4 (Montesquiu)</i>", tooltip=tooltip, icon=folium.Icon(icon="umbrella", prefix='fa')
    ).add_to(m)
    folium.Marker(
        [41.88244, 2.21483 ], popup="<i>VN (Vilobí d'Onyar)</i>", tooltip=tooltip, icon=folium.Icon(icon="umbrella", prefix='fa')
    ).add_to(m)
    folium.Marker(
        [41.84008, 2.41877 ], popup="<i>WS (Viladrau)</i>", tooltip=tooltip, icon=folium.Icon(icon="umbrella", prefix='fa')
      ).add_to(m)
    folium.Marker(
        [42.03947, 2.11993 ], popup="<i>V5 (Perafita)</i>", tooltip=tooltip, icon=folium.Icon(icon="umbrella", prefix='fa')
    ).add_to(m)
    
#     #Dades ACA (les coordenades están en format UTM, truquem una funció per pasarles a longitud - latitud)
    folium.Marker(
        utm.to_latlon(439144, 4655539, 31, 'northern'), popup="<i>Torrelló-Ges (F009891) </i>", tooltip=tooltip, icon=folium.Icon(color='red', icon="dashboard", prefix='fa')
    ).add_to(m)
    folium.Marker(
        utm.to_latlon(442102, 4648259, 31, 'northern'), popup="<i>Masies de Roda (L08116-72-00002) </i>", tooltip=tooltip, icon=folium.Icon(color='red', icon="dashboard", prefix='fa')
    ).add_to(m)
    folium.Marker(
        utm.to_latlon(439144, 4669209, 31, 'northern'), popup="<i>Ripoll (L17147-72-00005) </i>", tooltip=tooltip, icon=folium.Icon(color='red', icon="dashboard", prefix='fa')
    ).add_to(m)
    folium.Marker(
        utm.to_latlon(437546, 4674665, 31, 'northern'), popup="<i>Sant Joan de les Abadesses (L17167-72-00001) </i>", tooltip=tooltip, icon=folium.Icon(color='red', icon="dashboard", prefix='fa')
    ).add_to(m)
    folium.Marker(
        utm.to_latlon(469379, 4645530, 31, 'northern'), popup="<i>Anglès (F014672) </i>", tooltip=tooltip, icon=folium.Icon(color='red', icon="dashboard", prefix='fa')
    ).add_to(m)
    folium.Marker(
        utm.to_latlon(477269, 4651536, 31, 'northern'), popup="<i>Ginestar-Llemena (F000005) </i>", tooltip=tooltip, icon=folium.Icon(color='red', icon="dashboard", prefix='fa')
    ).add_to(m)
    folium.Marker(
        utm.to_latlon(484973, 4638465, 31, 'northern'), popup="<i>Campllong-Gotarra (L17038-72-00002)</i>", tooltip=tooltip, icon=folium.Icon(color='red', icon="dashboard", prefix='fa')
    ).add_to(m)
    folium.Marker(
        utm.to_latlon(485464, 4647022, 31, 'northern'), popup="<i>Girona-Onyar (L17079-72-00004) </i>", tooltip=tooltip, icon=folium.Icon(color='red', icon="dashboard", prefix='fa')
    ).add_to(m)
    folium.Marker(
        utm.to_latlon(484777, 4638503, 31, 'northern'), popup="<i>Riudellots-Onyar (F026458) </i>", tooltip=tooltip, icon=folium.Icon(color='red', icon="dashboard", prefix='fa')
    ).add_to(m)
    folium.Marker(
        utm.to_latlon(499269, 4658310, 31, 'northern'), popup="<i>Colomers (L17055-72-00002 	) </i>", tooltip=tooltip, icon=folium.Icon(color='red', icon="dashboard", prefix='fa')
    ).add_to(m)
    folium.Marker(
        utm.to_latlon(439144, 4648705, 31, 'northern'), popup="<i>Girona-Ter (L17079-72-00005) </i>", tooltip=tooltip, icon=folium.Icon(color='red', icon="dashboard", prefix='fa')
    ).add_to(m)
    folium.Marker(
        utm.to_latlon(467132, 4648235, 31, 'northern'), popup="<i>Pasteral-Cabal (F001242) </i>", tooltip=tooltip, icon=folium.Icon(color='red', icon="dashboard", prefix='fa')
    ).add_to(m)
    folium.Marker(
        utm.to_latlon(510378, 4653688, 31, 'northern'), popup="<i>Torroella de Montgrí (L17199-72-00001) </i>", tooltip=tooltip, icon=folium.Icon(color='red', icon="dashboard", prefix='fa')
    ).add_to(m)
    folium.Marker(
        utm.to_latlon(483379, 4660993, 31, 'northern'), popup="<i>Conellà Terri (F001243) </i>", tooltip=tooltip, icon=folium.Icon(color='red', icon="dashboard", prefix='fa')
    ).add_to(m)
    
    #Outline del Riu Ter
    folium.GeoJson(json.load(open("ter.geojson")), name="Riu Ter").add_to(m)
    
    return m

hide_toggle()

In [27]:
m = prepareMap()
m

# Analisis Outliers

In [18]:
def calcularOutlier(df, col, cont=0.005):
    #Crear una versio horaria del dataset
    df['timestamp'] = pd.to_datetime(df.index)
    df = df.set_index('timestamp').resample("H").mean().reset_index()
    
    #Usem el metode de Isolation Forest per trobar anomalies a les precipitacions
    output = df[[col, 'timestamp']]
    temp = df[[col, 'timestamp']]
    model=IsolationForest(n_estimators=50, max_samples='auto', contamination=cont,max_features=1.0)
    model.fit(temp[[col]].values)
    output['outliers']=pd.Series(model.predict(temp[[col]].values)).apply(lambda x: 'yes' if (x == -1) else 'no' )
    output.rename(columns={col:'data'}, inplace=True)
    
    return output

In [6]:
# Carregar dades pluja
Df_SMC_Total = pd.read_excel("finalsDF/DF_SMC.xlsx", index_col=0)
df = Df_SMC_Total

#['CC', 'CG', 'CI', 'CY', 'DG', 'DJ', 'DN', 'KE', 'UB', 'UN', 'UO', 'V3','V4', 'V5', 'VN', 'WS', 'DM_XJ_WF', 'Z4_ZC']

In [11]:
# Carregar dades rius
altTer = pd.read_excel("Dataframes/df_imputedAltTerKNN.xlsx", index_col=0)
baixTer = pd.read_excel("Dataframes/df_imputedBaixTerKNN.xlsx", index_col=0)


,data,timestamp,outliers
0,0.0,2009-01-01 00:00:00,no
1,0.0,2009-01-01 01:00:00,no
2,0.0,2009-01-01 02:00:00,no
3,0.0,2009-01-01 03:00:00,no
4,0.0,2009-01-01 04:00:00,no
...,...,...,...
113948,0.0,2021-12-31 20:00:00,no
113949,0.0,2021-12-31 21:00:00,no
113950,0.0,2021-12-31 22:00:00,no
113951,0.0,2021-12-31 23:00:00,no


In [8]:
altTer = pd.read_excel("Dataframes/df_imputedAltTerKNN.xlsx", index_col=0)
baixTer = pd.read_excel("Dataframes/df_imputedBaixTerKNN.xlsx", index_col=0)

#Outliers de l'estacio de ACA més propera riu avall de l'estacio metereologica CC
#torrello = calcularOutlier(baixTer, "L17079-72-00005")


In [9]:
#Mostrar en un plot les anomalies que trobem amb la precipitació a CC i el cabal a torrelló de 2019 a 2022
cc = cc.loc[cc['timestamp'] >= '2019-01-01 00:00:00']
torrello = torrello.loc[torrello['timestamp'] >= '2019-01-01 00:00:00']
fig = px.scatter(cc, x='timestamp', y='CC', color='outliers',title='Outlier CC')
fig.update_xaxes(
    rangeslider_visible=True,
)
fig2 = px.scatter(torrello, x='timestamp', y='L17079-72-00005', color='outliers',title='Outlier Torrello')
fig2.update_xaxes(
    rangeslider_visible=True,
)
fig.show()
fig2.show()

NameError: name 'torrello' is not defined

Veiem que tenim al 23 de gener de 2020 una anomalia molt notable, al buscar aquesta data a Google més les paraules "riu ter inundacio" sorten noticias sobre com va haver-hi grans inundacions per la zona del baix Ter, per tant podem comprovar que aquesta anomalia va ocurrir realment i no es tracta d'un error tècnic.

Noticia: https://cat.elpais.com/cat/2020/01/23/catalunya/1579810582_789729.html

Al gràfic de l'estació CC veiem que a les dates de 22-23 de gener si va a haver-hi outliers, pero en cuant a m3/s no son tan desproprocionats amb la resta de valors comparat amb les dades de Torrelló. (Lo que si veiem es molta freqüencia de outliers, to-do: potser comprovar importancia de freqüencia de pluja, en lloc de plujes extremadament fortes?)

Anem a veure si una mica més riu a munt podem trobar anomalies per aquesta mateixa data

In [ ]:
#Mirem una mica més amunt del riu, l'estacio meteorologica V4 i un altre cop l'estacio de ACA de torrelló

v4 = calcularOutlier(df, 'V4')
v4 = v4.loc[v4['timestamp'] >= '2019-01-01 00:00:00']
torrello = torrello.loc[torrello['timestamp'] >= '2019-01-01 00:00:00']
fig = px.scatter(v4, x='timestamp', y='V4', color='outliers',title='Outlier V4')
fig.update_xaxes(
    rangeslider_visible=True,
)
fig2 = px.scatter(torrello, x='timestamp', y='L17079-72-00005', color='outliers',title='Outlier Torrello', trendline='lowess')
fig2.update_xaxes(
    rangeslider_visible=True,
)
fig.show()
fig2.show()

Trobem dades molt similars amb l'estació metereologica CC, hi ha outliers a la mateixa data en la que van ocurrir les inundacions, pero la quantitat de pluja no es tan extremadament superior a la resta de outliers, como la quantitat de cabal durant aquesta data es a la resta de outliers. Lo que també veiem, es que van ocurrir les plujes amb molta frequencia durant aquesta data, mentre que a altres dies que també tenen outliers, els enregistraments d'aqusts outliers están més espaciats en el temps.

Mirem ara amb més detall durant la data que van haver-hi les inundacions, mirant també les dades de ACA a la part més amunt del riu i a la desembocadura.

In [ ]:
start_date = widgets.DatePicker(
    description='Data Inici',
    disabled=False
)
end_date = widgets.DatePicker(
    description='Data Final',
    disabled=False
)

widgets.HBox([start_date, end_date])

In [10]:
cc = calcularOutlier(df, 'CC')
torrello = calcularOutlier(baixTer, "L17079-72-00005")
santJoan = calcularOutlier(altTer, "L17167-72-00001")
massiesRodar = calcularOutlier(altTer, "L08116-72-00002")
torroellaMontegri = calcularOutlier(baixTer, "L17199-72-00001")
v4 = calcularOutlier(df, 'V4')

C:\Users\34605\AppData\Local\Temp\ipykernel_18060\1817129576.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\34605\AppData\Local\Temp\ipykernel_18060\1817129576.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\34605\AppData\Local\Temp\ipykernel_18060\1817129576.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [22]:
start_date = pd.to_datetime(start_date.value)
end_date = pd.to_datetime(end_date.value)

cc = cc.loc[(cc['timestamp'] >= start_date) & (cc['timestamp'] <= end_date)]
v4 = v4.loc[(v4['timestamp'] >= start_date) &  (v4['timestamp'] <= end_date)]
torrello = torrello.loc[(torrello['timestamp'] >= start_date) & (torrello['timestamp'] <= end_date)]
santJoan = santJoan.loc[(santJoan['timestamp'] >= start_date) & (santJoan['timestamp'] <= end_date)]
torroellaMontegri = torroellaMontegri.loc[(torroellaMontegri['timestamp'] >= start_date) & (torroellaMontegri['timestamp'] <= end_date)]

fig = px.scatter(cc, x='timestamp', y='CC', color='outliers',title='Outlier CC')
fig.update_xaxes(
    rangeslider_visible=True,
)
fig2 = px.scatter(v4, x='timestamp', y='V4', color='outliers',title='Outlier V4')
fig2.update_xaxes(
    rangeslider_visible=True,
)
fig3 = px.scatter(santJoan, x='timestamp', y='L17167-72-00001', color='outliers',title='Outlier Sant Joan (Neixement)')
fig3.update_xaxes(
    rangeslider_visible=True,
)
fig4 = px.scatter(torrello, x='timestamp', y='L17079-72-00005', color='outliers',title='Outlier Torrello')
fig4.update_xaxes(
    rangeslider_visible=True,
)
fig5 = px.scatter(torroellaMontegri, x='timestamp', y='L17199-72-00001', color='outliers',title='Outlier Torroella-Montegri (Desembocadura)')
fig5.update_xaxes(
    rangeslider_visible=True,
)
fig.show()
fig2.show()
fig3.show()
fig4.show()
fig5.show()

Veiem que a totes les estacions del riu hi ha una pujada de cabal, seguida al dia seguent per una pujada encara mes gran. En canvi a la pluja es a l'inrevés, el primer dia plou més que al segon dia (on el cabal es més alt).

Anem a comprovar la pluja al neixement del riu.

In [46]:
start_date = pd.to_datetime(start_date.value)
end_date = pd.to_datetime(end_date.value)

cg = calcularOutlier(df, 'CG')
dg = calcularOutlier(df, 'DG')


cg = cg.loc[(cg['timestamp'] >= start_date) & (cg['timestamp'] <= end_date)]
dg = dg.loc[(dg['timestamp'] >= start_date) &  (dg['timestamp'] <= end_date)]

cg['CG'] = cg['CG'].rolling(window=5).mean()
dg['DG'] = dg['DG'].rolling(window=5).mean()

fig = px.scatter(cg, x='timestamp', y='CG', color='outliers',title='Outlier CG')
fig.update_xaxes(
    rangeslider_visible=True,
)
fig2 = px.scatter(dg, x='timestamp', y='DG', color='outliers',title='Outlier DG')
fig2.update_xaxes(
    rangeslider_visible=True,
)

fig.show()
fig2.show()

/var/folders/d4/kxqcphy54q9g3rr5tlcvhv9h0000gn/T/ipykernel_60869/1817129576.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/d4/kxqcphy54q9g3rr5tlcvhv9h0000gn/T/ipykernel_60869/1817129576.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/d4/kxqcphy54q9g3rr5tlcvhv9h0000gn/T/ipykernel_60869/2756019333.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [53]:
pasteral = calcularOutlier(baixTer, 'F001242')

/var/folders/d4/kxqcphy54q9g3rr5tlcvhv9h0000gn/T/ipykernel_60869/1817129576.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [55]:
pasteral = pasteral.loc[(pasteral['timestamp'] >= start_date) &  (pasteral['timestamp'] <= end_date)]
fig = px.scatter(pasteral, x='timestamp', y='F001242', color='outliers',title='Anomaly Pasteral')
fig.update_xaxes(
    rangeslider_visible=True,
)

fig.show()

In [32]:
cc

,CC,timestamp,outliers
96864,0.0,2020-01-20 00:00:00,no
96865,0.0,2020-01-20 01:00:00,no
96866,0.0,2020-01-20 02:00:00,no
96867,0.0,2020-01-20 03:00:00,no
96868,0.0,2020-01-20 04:00:00,no
...,...,...,...
97004,0.0,2020-01-25 20:00:00,no
97005,0.0,2020-01-25 21:00:00,no
97006,0.0,2020-01-25 22:00:00,no
97007,0.0,2020-01-25 23:00:00,no


Utilitzem el metode de crosscorrelation per comprovar a partir de quin lag (en aquest cas diferencia de horas) hi ha més correlacio entre el cabal en distintes parts del riu

## A 1 hora

In [59]:
sm.tsa.stattools.ccf(torrello['L17079-72-00005'], santJoan['L17167-72-00001'], adjusted=False)

array([ 0.58337159,  0.6320624 ,  0.67921972,  0.72151541,  0.75632764,
        0.78215156,  0.79877282,  0.80608162,  0.80281016,  0.78990122,
        0.77068964,  0.74768201,  0.72383521,  0.70078475,  0.67827103,
        0.65626913,  0.63399769,  0.61033273,  0.58356577,  0.55366027,
        0.52217667,  0.49090656,  0.46149567,  0.43541972,  0.41330267,
        0.39562536,  0.38300417,  0.37515833,  0.37211013,  0.37343036,
        0.37794257,  0.38448176,  0.39110717,  0.39323395,  0.3882394 ,
        0.37529061,  0.35211736,  0.31853388,  0.27674043,  0.23075314,
        0.18400653,  0.13860649,  0.09582211,  0.05434528,  0.01320208,
       -0.02897947, -0.07220481, -0.11583376, -0.15953271, -0.20196961,
       -0.24222925, -0.27791662, -0.30783832, -0.33345131, -0.35418189,
       -0.36897559, -0.37753906, -0.3811076 , -0.38121064, -0.37905489,
       -0.37731603, -0.37817562, -0.38005783, -0.38274801, -0.38665485,
       -0.38998672, -0.38993124, -0.38867948, -0.38690258, -0.38

In [56]:
sm.tsa.stattools.ccf(pasteral['F001242'], torrello['L17079-72-00005'], adjusted=False)

array([ 0.8299975 ,  0.80264428,  0.77007479,  0.7348071 ,  0.69907221,
        0.66400097,  0.62939939,  0.59483321,  0.56085135,  0.5277577 ,
        0.49599604,  0.46576401,  0.43766906,  0.41180342,  0.38811368,
        0.36720254,  0.35012554,  0.33815232,  0.33179455,  0.33093741,
        0.33490211,  0.34328922,  0.3551242 ,  0.36820574,  0.38094982,
        0.39075645,  0.39466436,  0.39112644,  0.37966444,  0.36109381,
        0.33637028,  0.30668893,  0.27327127,  0.23721163,  0.19986556,
        0.16244509,  0.12674796,  0.09419497,  0.06566299,  0.04092994,
        0.0195978 ,  0.00084581, -0.01617156, -0.03232617, -0.04812721,
       -0.06356067, -0.07877548, -0.09368313, -0.10811646, -0.12226036,
       -0.1365814 , -0.15167055, -0.16786254, -0.18504611, -0.20291732,
       -0.22124697, -0.2391513 , -0.25626298, -0.27209037, -0.28638075,
       -0.30017462, -0.3137527 , -0.32738511, -0.34085331, -0.3509768 ,
       -0.35950344, -0.36809912, -0.37685044, -0.38514855, -0.39

In [62]:
sm.tsa.stattools.ccf(torrello['L17079-72-00005'], torroellaMontegri['L17199-72-00001'], adjusted=False)

array([ 0.77129141,  0.73145427,  0.69090743,  0.65118521,  0.61373684,
        0.57956871,  0.54964557,  0.52446749,  0.50436393,  0.48923536,
        0.47871471,  0.4725935 ,  0.47081954,  0.4731863 ,  0.47929326,
        0.48811932,  0.49757798,  0.50538947,  0.50951265,  0.50837186,
        0.50078733,  0.48626084,  0.46520212,  0.43872936,  0.40759481,
        0.37157866,  0.33083965,  0.28586216,  0.23838922,  0.1905552 ,
        0.14390688,  0.10021226,  0.06042346,  0.02384348, -0.00968081,
       -0.03932855, -0.06456265, -0.08574384, -0.10273802, -0.11470756,
       -0.12113158, -0.1228864 , -0.12082204, -0.11737233, -0.11498316,
       -0.11436747, -0.11621121, -0.12080412, -0.12745087, -0.13526183,
       -0.14379553, -0.15151867, -0.1576655 , -0.16443077, -0.17188813,
       -0.17993936, -0.18828028, -0.1967742 , -0.20503351, -0.21431627,
       -0.22847284, -0.24927949, -0.27344428, -0.29820367, -0.3229671 ,
       -0.3466895 , -0.36869924, -0.38813803, -0.4045819 , -0.41

In [66]:
sm.tsa.stattools.ccf(torrello['L17079-72-00005'],cc['CC'], adjusted=False)

array([ 0.00389266,  0.03749214,  0.06534445,  0.09047427,  0.12600043,
        0.16563985,  0.19828371,  0.22466531,  0.24152937,  0.25148631,
        0.24753018,  0.22495878,  0.20182628,  0.17718803,  0.14619827,
        0.11998211,  0.10281864,  0.09352748,  0.09429569,  0.1013704 ,
        0.11488973,  0.13426781,  0.15679908,  0.18146472,  0.20616791,
        0.235374  ,  0.27488248,  0.32135822,  0.36779612,  0.40506147,
        0.42456289,  0.44292367,  0.46208772,  0.47080787,  0.47834493,
        0.49109662,  0.5001917 ,  0.5065718 ,  0.49456034,  0.47297221,
        0.44432904,  0.41084673,  0.37936543,  0.34125777,  0.30558098,
        0.27215857,  0.23981107,  0.21514827,  0.191838  ,  0.17810852,
        0.16996133,  0.16099818,  0.1570893 ,  0.15510275,  0.15494684,
        0.15031588,  0.1483142 ,  0.14498412,  0.14615369,  0.14201621,
        0.12760995,  0.11619528,  0.10623665,  0.08833451,  0.06283985,
        0.03491329,  0.02306208, -0.00370324, -0.03303289, -0.04

## Funció Propia Correlació

In [19]:
santJoan = calcularOutlier(altTer, "L17167-72-00001")
massiesRoda = calcularOutlier(altTer, "L08116-72-00002")

start_date = '2020/01/20'
end_date = '2020/01/26'
santJoan = santJoan.loc[(santJoan['timestamp'] >= start_date) & (santJoan['timestamp'] <= end_date)]
massiesRoda = massiesRoda.loc[(massiesRoda['timestamp'] >= start_date) & (massiesRoda['timestamp'] <= end_date)]


C:\Users\34605\AppData\Local\Temp\ipykernel_18060\1817129576.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\34605\AppData\Local\Temp\ipykernel_18060\1817129576.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [18]:
r, p = stats.pearsonr(santJoan['L17167-72-00001'], massiesRoda['L08116-72-00002'])
print(f"Pearson r: {r} and p-value: {p}")

Pearson r: 0.8817639902852287 and p-value: 1.623020879226671e-48


In [31]:
def pearson_forward_lag(x, y, n=40):
    results = []
    offset = 0
    while offset < n:
        r, p = stats.pearsonr(x[0:n], y[offset:offset+n])
        print(f"Time Lag: {offset} , Pearson r: {r} and p-value: {p}")
        results.append({offset:(r,p)})
        offset += 1
        
    return results
        



In [32]:
x = santJoan['L17167-72-00001']
y = massiesRoda['L08116-72-00002']
pearson_forward_lag (x,y)

Time Lag: 0 , Pearson r: 0.8859111775642543 and p-value: 3.0296585605759824e-14
Time Lag: 1 , Pearson r: 0.9360131221578123 and p-value: 8.00519620956184e-19
Time Lag: 2 , Pearson r: 0.9309062721487784 and p-value: 3.291738843463257e-18
Time Lag: 3 , Pearson r: 0.8838039706427429 and p-value: 4.2087637018445344e-14
Time Lag: 4 , Pearson r: 0.8378552148746783 and p-value: 1.5545152944929802e-11
Time Lag: 5 , Pearson r: 0.7730955065235265 and p-value: 5.021681916153748e-09
Time Lag: 6 , Pearson r: 0.7266724364452494 and p-value: 1.1030359113920029e-07
Time Lag: 7 , Pearson r: 0.7109164149248497 and p-value: 2.742171215228525e-07
Time Lag: 8 , Pearson r: 0.7166551979512709 and p-value: 1.981927336247917e-07
Time Lag: 9 , Pearson r: 0.7425418316378288 and p-value: 4.129842996648707e-08
Time Lag: 10 , Pearson r: 0.7843128708242615 and p-value: 2.131834288447623e-09
Time Lag: 11 , Pearson r: 0.8326246607642173 and p-value: 2.7076509795825113e-11
Time Lag: 12 , Pearson r: 0.8622456424236469 a

[{0: (0.8859111775642543, 3.0296585605759824e-14)},
 {1: (0.9360131221578123, 8.00519620956184e-19)},
 {2: (0.9309062721487784, 3.291738843463257e-18)},
 {3: (0.8838039706427429, 4.2087637018445344e-14)},
 {4: (0.8378552148746783, 1.5545152944929802e-11)},
 {5: (0.7730955065235265, 5.021681916153748e-09)},
 {6: (0.7266724364452494, 1.1030359113920029e-07)},
 {7: (0.7109164149248497, 2.742171215228525e-07)},
 {8: (0.7166551979512709, 1.981927336247917e-07)},
 {9: (0.7425418316378288, 4.129842996648707e-08)},
 {10: (0.7843128708242615, 2.131834288447623e-09)},
 {11: (0.8326246607642173, 2.7076509795825113e-11)},
 {12: (0.8622456424236469, 8.78195903153901e-13)},
 {13: (0.887544400560812, 2.3378020674477106e-14)},
 {14: (0.8948707199122306, 6.942741191484665e-15)},
 {15: (0.8808225279693077, 6.630032812373847e-14)},
 {16: (0.8572004625635627, 1.661102925270142e-12)},
 {17: (0.7930731381081568, 1.0533317190950546e-09)},
 {18: (0.7098400985434252, 2.9118338178076567e-07)},
 {19: (0.61140076